# 401 First guess for long-term CDR needs

In this notebook, we create a lookup table to get a first estimate of the CDR needs for drawdown.

In [1]:
import pyam 
import pandas as pd
import numpy as np

import dotenv
import os
from pathlib import Path

%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Step 1: Read in the lookup table generated in the notebook `304_compile_metrics.ipynb`

In [2]:
metrics = pd.read_csv(
    Path(
        '../data/304_compiled_metrics.csv'
    )
)

<IPython.core.display.Javascript object>

In [3]:
metrics.head(2)

,model,scenario,run_id,drawdown_co2,ZEC,change_non_co2,2015_warming,2100_warming,peak_warming,year_peak_warming,cum_emissions_to_peak_GtCO2,cum_emissions_after_peak_GtCO2,eTCREup,eTCREdown
0,COFFEE 1.1,EN_NPi2020_400f_lowBECCS,0,0.173288,-0.034173,0.029602,1.118903,1.742949,1.931014,2059,983.820216,-352.723668,0.000825,-0.000491
1,COFFEE 1.1,EN_NPi2020_400f_lowBECCS,1,0.106326,-0.013109,0.013666,1.412041,2.977056,3.097214,2060,983.807875,-353.833081,0.001713,-0.000300


<IPython.core.display.Javascript object>

In [4]:
metrics.set_index(
    [
        'model',
        'scenario',
        'run_id'
    ],
    inplace=True
)

<IPython.core.display.Javascript object>

Step 2: Calculate the desired long-term cooling necessary to hit 1.5 in 2100. For this, we calculate the difference between 1.5 and the 2100 warming.

In [5]:
metrics.loc[:,'cooling_to_1p5'] = (
    1.5
    -
    metrics.loc[:, '2100_warming']
)

<IPython.core.display.Javascript object>

Step 3: Some of these may be positive because peak warming is below 1.5. We want to set these estimates to 0.

In [6]:
metrics.loc[:, 'cooling_to_1p5'] = (
    metrics.loc[:,'cooling_to_1p5'].apply(
        lambda x: x if x < 0 else 0
    )
)

<IPython.core.display.Javascript object>

Step 4: We can now calculate the first guess of the amount of cdr that may be required. However, the eTCREdown does not always show a "cooling" sign. To address this, we use a two tiered approach:
1. Use eTCREdown if it is negative
2. Use - (eTCREup) if eTCREdown is positive
Since this is only for a first guess, we can assume that this error will be corrected for in our iterative procedure later on.

In [7]:
metrics.loc[:, 'eTCREdown_first_guess'] = (
    metrics.apply(
        lambda x: x['eTCREdown'] if x['eTCREdown'] < 0 else -x['eTCREup'],
        axis=1
    )
)

<IPython.core.display.Javascript object>

Step 5: Calculate the additional CDR needs (first guess)

In [8]:
metrics.loc[:, 'additional_cdr_gtco2_first_guess'] = (
    metrics.loc[:, 'cooling_to_1p5']
    /
    metrics.loc[:, 'eTCREdown_first_guess'] 
).round(0)

<IPython.core.display.Javascript object>

In [9]:
metrics

drawdown_co2  \
model                 scenario                      run_id                 
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0           0.173288   
                                                    1           0.106326   
                                                    2           0.180099   
                                                    3           0.148464   
                                                    4           0.162020   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0           0.039350   
                                                    1           0.053018   
                                                    2           0.040664   
                                                    3           0.032436   
                                                    4           0.037088   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0           0.031644   
                                                    1          -0.002678   
                                                    2           0.031482   
                                                    3           0.026323   
                                                    4           0.030057   

                                                                 ZEC  \
model                 scenario                      run_id             
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0      -0.034173   
                                                    1      -0.013109   
                                                    2      -0.025046   
                                                    3      -0.063790   
                                                    4      -0.037898   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0      -0.036442   
                                                    1      -0.013612   
                                                    2      -0.023099   
                                                    3      -0.065566   
                                                    4      -0.042074   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0      -0.041529   
                                                    1      -0.008364   
                                                    2      -0.038951   
                                                    3      -0.075703   
                                                    4      -0.047712   

                                                            change_non_co2  \
model                 scenario                      run_id                   
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0             0.029602   
                                                    1             0.013666   
                                                    2             0.013126   
                                                    3            -0.026165   
                                                    4             0.022756   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0            -0.237951   
                                                    1            -0.142832   
                                                    2            -0.232387   
                                                    3            -0.279659   
                                                    4            -0.334983   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0            -0.023157   
                                                    1            -0.007957   
                                                    2             0.001200   
                                                    3            -0.063334   
                                                    4            -0.049273   

                                                            2015_warming  \
model                 scenario                      run_id                 
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0          

<IPython.core.display.Javascript object>

Step 6: We probably want to add in the year of net zero CO2.

In [10]:
meta = pd.read_excel(
    Path(
        '../data/100_meta.xlsx'
    ),
    index_col=[0,1]
)

<IPython.core.display.Javascript object>

In [11]:
metrics.loc[:, 'netzero|CO2'] = (
    metrics.apply(
        lambda x: meta.loc[x.name[0:2], 'netzero|CO2'],
        axis=1
    )
)

<IPython.core.display.Javascript object>

Step 7: Write this out for further assessment.

In [12]:
metrics

drawdown_co2  \
model                 scenario                      run_id                 
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0           0.173288   
                                                    1           0.106326   
                                                    2           0.180099   
                                                    3           0.148464   
                                                    4           0.162020   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0           0.039350   
                                                    1           0.053018   
                                                    2           0.040664   
                                                    3           0.032436   
                                                    4           0.037088   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0           0.031644   
                                                    1          -0.002678   
                                                    2           0.031482   
                                                    3           0.026323   
                                                    4           0.030057   

                                                                 ZEC  \
model                 scenario                      run_id             
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0      -0.034173   
                                                    1      -0.013109   
                                                    2      -0.025046   
                                                    3      -0.063790   
                                                    4      -0.037898   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0      -0.036442   
                                                    1      -0.013612   
                                                    2      -0.023099   
                                                    3      -0.065566   
                                                    4      -0.042074   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0      -0.041529   
                                                    1      -0.008364   
                                                    2      -0.038951   
                                                    3      -0.075703   
                                                    4      -0.047712   

                                                            change_non_co2  \
model                 scenario                      run_id                   
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0             0.029602   
                                                    1             0.013666   
                                                    2             0.013126   
                                                    3            -0.026165   
                                                    4             0.022756   
REMIND-MAgPIE 2.1-4.2 SusDev_SDP-PkBudg1000         0            -0.237951   
                                                    1            -0.142832   
                                                    2            -0.232387   
                                                    3            -0.279659   
                                                    4            -0.334983   
REMIND-MAgPIE 2.1-4.3 DeepElec_SSP2_ HighRE_Budg900 0            -0.023157   
                                                    1            -0.007957   
                                                    2             0.001200   
                                                    3            -0.063334   
                                                    4            -0.049273   

                                                            2015_warming  \
model                 scenario                      run_id                 
COFFEE 1.1            EN_NPi2020_400f_lowBECCS      0          

<IPython.core.display.Javascript object>

In [13]:
metrics.to_csv(
    Path(
        '../data/401_lookup.csv'
    )
)

<IPython.core.display.Javascript object>